In [1]:
import datasets

with open("polys.csv") as f:
    next(f)
    data = list(x.replace(" ", "").strip() for x in f)

ds = datasets.Dataset.from_dict({"input": data})
ds = ds.map(lambda x: {"input_ids": [ord(c) for c in x["input"]]}, remove_columns=ds.column_names)
# ds = ds.filter(lambda x: len(x["input_ids"]) < 4000)
# def pad(x):
#     l = max(len(a) for a in x["input_ids"])
#     return {"input_ids": [a + ([0] * (l - len(a) + 1)) for a in x["input_ids"]]}
# ds = ds.map(pad, batched=True, batch_size=128)
ds = ds.train_test_split(test_size=1000, seed=7)
ds.save_to_disk("./polys")

  0%|          | 0/1835007 [00:00<?, ?ex/s]

KeyboardInterrupt: 

In [ ]:
!pip install -U transformers accelerate

In [ ]:
import datasets

In [ ]:
ds = datasets.load_from_disk("./polys")
ds = ds.map(lambda x: {"prefix_len": len(x["input_ids"]) - 1 - x["input_ids"][::-1].index(ord(','))})

In [36]:
# inp = ds["test"][11]["input_ids"][:50]
def test(s):
    model.eval()
    def de(x): return "".join([chr(y) for y in x])
    inp = [ord(x) for x in s]
    out = model.model.generate(torch.tensor([inp]).cuda(), max_new_tokens=50, num_beams=20, pad_token_id=0, do_sample=False)
    # print(de(inp))
    print(de(out[0]))
    model.train()
test("1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,")
test("9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,")
test("4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,")
test("1,8,255,3280,21845,97656,335923,960800,2396745,5380840,11111111,21435888,39089245,67977560,113522235,183063616,286331153,435984840,648232975,943531280,")

1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,                                                  
9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,RRZR0S0SZ0S0SSSSZX                                
4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,RRZR0S0SZ0S0SSSSZX                                
1,8,255,3280,21845,97656,335923,960800,2396745,5380840,11111111,21435888,39089245,67977560,113522235,183063616,286331153,435984840,648232975,943531280,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSSZ1


In [ ]:
ds

In [ ]:
import transformers, dataclasses, torch


config = transformers.GPTJConfig(
    vocab_size=256,
    n_embd=256,
    rotary_dim=16,
    n_layer=12,
    n_head=4,
    n_positions=4096,
    resid_pdrop=0.1,
    embd_pdrop=0.1,
    attn_pdrop=0.1
)
# model = transformers.GPTJForCausalLM(config).cuda()
# model = transformers.GPTJForCausalLM.from_pretrained("./polys-model-2/checkpoint-20000/").cuda()

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = transformers.GPTJForCausalLM(config).cuda()
        # self.model = transformers.GPTJForCausalLM.from_pretrained("./polys-model-4-dropout-2/checkpoint-50000/").cuda()
    def forward(self, labels, prefix_len, input_ids):
        out = self.model(input_ids.cuda())
        lm_logits = out.logits
        # # move labels to correct device to enable model parallelism
        labels = labels.to(lm_logits.device)
        # Shift so that tokens < n predict n
        shift_logits = lm_logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        # Flatten the tokens
        loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
        loss = loss_fct(shift_logits.permute(0, 2, 1), shift_labels)
        seq_dim = loss.shape[1]
        r = torch.arange(seq_dim).expand(8, -1).cuda()
        loss_mask = (r < (prefix_len + 1).unsqueeze(1).expand(-1, seq_dim))
        loss[loss_mask] = 0
        return dataclasses.replace(out, loss=loss.mean())

model = Model()
model.load_state_dict(torch.load("./polys-model-4-dropout-2-cont/checkpoint-100000/pytorch_model.bin"))

In [6]:
sum(dict((p.data_ptr(), p.numel()) for p in model.parameters()).values()) # model num parameters

9590528

In [ ]:
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./polys-model-4-dropout-2-cont",
    overwrite_output_dir=True,
    num_train_epochs=1,
    # per_gpu_train_batch_size=64,
    save_steps=10_000,
    # save_total_limit=2,
    fp16=True,
    evaluation_strategy="steps",
    eval_steps=500,
    remove_unused_columns=False,
    # prediction_loss_only=True,
)

def collator(x):
    l = max(len(a["input_ids"]) for a in x)
    v = torch.tensor([a["input_ids"] + ([0] * (l - len(a["input_ids"]) + 1)) for a in x])
    # print(v.shape)
    # return {"input_ids": v, "labels": v}
    return {"input_ids": v, "labels": v, "prefix_len": torch.tensor([a["prefix_len"] for a in x])}

class Cb(transformers.TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        with torch.no_grad():
            model = kwargs["model"]
            model.eval()
            def de(x): return "".join([chr(y) for y in x])
            d = ds["test"][5]
            inp = d["input_ids"][:d["prefix_len"] + 1]
            out = model.model.generate(torch.tensor([inp]).cuda(), max_new_tokens=200, pad_token_id=0, eos_token_id=0)
            print("real:", de(d["input_ids"]))
            print("out: ", de(out[0]))
            inp = "1,1,2,6,24,120,720,5040,40320,362880,3628800,39916800,479001600,6227020800,87178291200,1307674368000,20922789888000,355687428096000,6402373705728000,121645100408832000,"
            inp = [ord(x) for x in inp]
            out = model.model.generate(torch.tensor([inp]).cuda(), max_new_tokens=200, pad_token_id=0, eos_token_id=0)
            print("fact:", de(out[0]))
            model.train()

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    callbacks=[Cb()]
)
trainer.train()

Using cuda_amp half precision backend
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1834007
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 229251
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: acertain. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
500,0.018100,0.016155
1000,0.017900,0.016253
1500,0.017400,0.015194
2000,0.017100,0.015925
2500,0.016700,0.017286
3000,0.016000,0.018549
3500,0.015200,0.012048
4000,0.013500,0.007532
4500,0.010900,0.006157
5000,0.008900,0.006623


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


real: 2,17,442,4277,22658,83977,246602,615197,1360642,2743553,5141402,9079237,15264002,24622457,38342698,57919277,85201922,122447857,172377722,238235093,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSZ10S0SZ8S0SSSSZ6S0SZ4S0SSSSZ2S0Z0S0SSZX
out:  2,17,442,4277,22658,83977,246602,615197,1360642,2743553,5141402,9079237,15264002,24622457,38342698,57919277,85201922,122447857,172377722,238235093,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSZ10S0SZ8S0SSSSZ6S0SSSSZ4S0SZ2S0Z0S0SSZX 
fact: 1,1,2,6,24,120,720,5040,40320,362880,3628800,39916800,479001600,6227020800,87178291200,1307674368000,20922789888000,355687428096000,6402373705728000,121645100408832000,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSSZ10S0SSSSZ8S0SZ6S0SZ4S0Z2S0Z0S0SZX 


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


...

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


real: 2,17,442,4277,22658,83977,246602,615197,1360642,2743553,5141402,9079237,15264002,24622457,38342698,57919277,85201922,122447857,172377722,238235093,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSZ10S0SZ8S0SSSSZ6S0SZ4S0SSSSZ2S0Z0S0SSZX
out:  2,17,442,4277,22658,83977,246602,615197,1360642,2743553,5141402,9079237,15264002,24622457,38342698,57919277,85201922,122447857,172377722,238235093,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSZ10S0SZ8S0SSSSZ6S0SZ4S0SSSSZ2S0Z0S0SSZX 
fact: 1,1,2,6,24,120,720,5040,40320,362880,3628800,39916800,479001600,6227020800,87178291200,1307674368000,20922789888000,355687428096000,6402373705728000,121645100408832000,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSSZ10S0Z8S0Z6S0SSSSSSZ4S0Z2S0Z0S0SZX 


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


real: 2,17,442,4277,22658,83977,246602,615197,1360642,2743553,5141402,9079237,15264002,24622457,38342698,57919277,85201922,122447857,172377722,238235093,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSZ10S0SZ8S0SSSSZ6S0SZ4S0SSSSZ2S0Z0S0SSZX
out:  2,17,442,4277,22658,83977,246602,615197,1360642,2743553,5141402,9079237,15264002,24622457,38342698,57919277,85201922,122447857,172377722,238235093,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSZ10S0SZ8S0SSSSZ6S0SZ4S0SSSSZ2S0Z0S0SSZX 
fact: 1,1,2,6,24,120,720,5040,40320,362880,3628800,39916800,479001600,6227020800,87178291200,1307674368000,20922789888000,355687428096000,6402373705728000,121645100408832000,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSSZ10S0Z8S0Z6S0SZ4S0SZ2S0SSSSSSZ0S0SZX 


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


In [ ]:

# epochs = (num_samples // train_loader.dataset.num_rows) + 1
# optim = torch.optim.AdamW()

#   wandb.init(project="page-quality", entity="acertain")
#   wandb.watch(model)

#   for n_epoch in tqdm(range(epochs)):
#    epoch = float(n_epoch)/epochs
#    # if n_epoch % 2 == 0:
#    #   embeddings[:,:] = 0
#    #   grad_embeddings[:,:] = 0
#    #   grad_embeddings_n[:] = 0
#    for (step,x) in enumerate(tqdm(train_loader)):
#       model.train()

#       # loss_bw, loss, loss_grad = run_model(x) # , modify_embs=False
#       model(x, epoch)['loss'].backward()

#       # optim.first_step(zero_grad=True)
#       # run_model(x)[0].backward()
#       # optim.second_step(zero_grad=True)

#       optim.step()
#       optim.zero_grad(set_to_none=True)

#       # tqdm.write(repr(loss))

#       if step % eval_every == 0:
#         model.eval()
#         with torch.no_grad():
#           # TODO: mean over iters or w/e
#           eval_steps = 0
#           eval_loss = 0
#           accuracy = 0
#           for x in eval_loader:
#             eval_steps += 1
#             labels = x['label'].to(device)
#             logits = model(x, epoch)['logits']
#             if use_coral_loss:
#               eval_loss += coral_loss(logits, levels_from_labelbatch(labels, num_classes = num_classes))
#               pred_labels = proba_to_label(F.sigmoid(logits))
#             else:
#               eval_loss += F.cross_entropy(logits, labels)
#               pred_labels = logits.argmax(dim=1)
#             accuracy += (pred_labels == labels).sum().float() / float( labels.size(0) )
#           eval_loss = eval_loss / eval_steps; accuracy = accuracy / eval_steps
#           tqdm.write("eval loss: {}, accuracy: {}".format(eval_loss, accuracy))
#           wandb.log({"loss/eval": eval_loss, "accuracy/eval": accuracy}, commit=False)

#   torch.save(model.state_dict(), "sentence_embedding_classifier.pt")



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1,1,2,6,24,120,720,5040,40320,362880,3628800,39916800,479001600,6227020800,87178291200,1307674368000,20922789888000,355687428096000,6402373705728000,121645100408832000,
1,1,2,6,24,120,720,5040,40320,362880,3628800,39916800,479001600,6227020800,87178291200,1307674368000,20922789888000,355687428096000,6402373705728000,121645100408832000,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSSZ16565656565616565610652525616561656565656565656566666666666622276266262266532266662626433333333334466664444443030222887666426477242224242430RZR0S0S0RZR0S0RZR0S0S0RZR0S0RZR0S0RZR0S0RRZR0S0SZR0S030S03030S03030SZR0S0S0S0SZR0S0SZR0S0SZRRZZZZZZZZZ5Z5S0S0SSSSZ56666652465246524S0SSS0S0SSSS0S0SSSS0S0SSSS0S0SSSSSSSSSSSSSSSZ6666666666666666653,1212S0SS


In [106]:
def de(x):
    return "".join([chr(y) for y in x])
de(ds["test"][0]["input_ids"])
"0,11,234,1641,6692,20055,49326,105749,204936,367587,620210,995841,1534764,2285231,3304182,4657965,6423056,8686779,11548026,15117977,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSSZ8S0SSZ6S0Z4S0SSZ2S0SZ0S0ZX"

'0,11,234,1641,6692,20055,49326,105749,204936,367587,620210,995841,1534764,2285231,3304182,4657965,6423056,8686779,11548026,15117977,RRZR0S0RRZR0S0RRZR0S0RRZR0S0RRZR0S0SSSSSSZ8S0SSZ6S0Z4S0SSZ2S0SZ0S0ZX'

In [47]:
len("1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1".split(","))

33

In [50]:
!pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 107.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 80.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 102.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.12.0
    Uninstalling huggingface-hub-0.12.0:
      Successfully uninstalled huggingface-hub-0.12.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.21.3
    Uninstalling transformers-4.21.3:
      Successfully uninstalled transformers-4.21.3


In [9]:
sum(dict((p.data_ptr(), p.numel()) for p in model.parameters()).values())


330752